# 使用BIDS数据格式

In [1]:
import matplotlib
import pathlib
import mne
import mne_bids

matplotlib.use('Qt5Agg')  # for use on clusters without X server

## 读取数据

In [23]:
# sample_data_dir = mne.datasets.sample.data_path(path='F:/Database/Multimodal_data/mne_data/MNE-sample-data/', download=False)
sample_data_dir = 'F:/Database/Multimodal_data/mne_data/MNE-sample-data/' # 设置已有数据路径
sample_data_dir = pathlib.Path(sample_data_dir)

raw_path = sample_data_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)

events = mne.find_events(raw)
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}

Opening raw data file F:\Database\Multimodal_data\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
320 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5 32]


## 将读取的原始数据转换为BIDS格式

In [24]:
# 首先需要指定电源线频率
raw.info['line_freq'] = 60  # specify power line frequency as required by ICA


In [29]:
# 写入BIDS格式
out_path = pathlib.Path(sample_data_dir / 'out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              root=out_path)

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events, event_id=event_id, overwrite=True)

Opening raw data file F:\Database\Multimodal_data\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Writing 'F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\participants.tsv'...
Writing 'F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\participants.json'...
Writing 'F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-02\ses-01\meg\sub-02_ses-01_coordsystem.json'...
Writing 'F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-02\ses-01\meg\sub-02_ses-01_coordsystem.json'...
Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Left', 'Visual/Right']
Writing 'F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-02

C:\Users\pomelo\AppData\Local\Temp\ipykernel_34000\3275398957.py:10: FutureWarning: The events_data parameter has been deprecated in favor the new events parameter, to ensure better consistency  with MNE-Python. The events_data parameter will be removed in MNE-BIDS 0.14. Please use the events parameter instead.
  mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events, event_id=event_id, overwrite=True)


Closing F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-02\ses-01\meg\sub-02_ses-01_task-audiovisual_run-01_meg.fif
[done]
Writing 'F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-02\ses-01\sub-02_ses-01_scans.tsv'...
Wrote F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-02\ses-01\sub-02_ses-01_scans.tsv entry with meg\sub-02_ses-01_task-audiovisual_run-01_meg.fif.


BIDSPath(
root: F:/Database/Multimodal_data/mne_data/MNE-sample-data/out_data/sample_BIDS
datatype: meg
basename: sub-02_ses-01_task-audiovisual_run-01_meg.fif)

## 编写MEGIN/Elekta/NeuroMag微调和串扰文件

In [28]:
cal_path = sample_data_dir / 'SSS' / 'sss_cal_mgh.dat'
ct_path = sample_data_dir / 'SSS' / 'ct_sparse_mgh.fif'

mne_bids.write_meg_calibration(cal_path, bids_path=bids_path)
mne_bids.write_meg_crosstalk(ct_path, bids_path=bids_path)

Writing fine-calibration file to F:/Database/Multimodal_data/mne_data/MNE-sample-data/out_data/sample_BIDS/sub-01/ses-01/meg/sub-01_ses-01_acq-calibration_meg.dat
Writing crosstalk file to F:/Database/Multimodal_data/mne_data/MNE-sample-data/out_data/sample_BIDS/sub-01/ses-01/meg/sub-01_ses-01_acq-crosstalk_meg.fif


## 查看创建的文件和目录结构

In [30]:
mne_bids.print_dir_tree(out_path)

|sample_BIDS\
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-01\
|------ ses-01\
|--------- sub-01_ses-01_scans.tsv
|--------- meg\
|------------ sub-01_ses-01_acq-calibration_meg.dat
|------------ sub-01_ses-01_acq-crosstalk_meg.fif
|------------ sub-01_ses-01_coordsystem.json
|------------ sub-01_ses-01_task-audiovisual_run-01_channels.tsv
|------------ sub-01_ses-01_task-audiovisual_run-01_events.json
|------------ sub-01_ses-01_task-audiovisual_run-01_events.tsv
|------------ sub-01_ses-01_task-audiovisual_run-01_meg.fif
|------------ sub-01_ses-01_task-audiovisual_run-01_meg.json
|--- sub-02\
|------ ses-01\
|--------- sub-02_ses-01_scans.tsv
|--------- meg\
|------------ sub-02_ses-01_coordsystem.json
|------------ sub-02_ses-01_task-audiovisual_run-01_channels.tsv
|------------ sub-02_ses-01_task-audiovisual_run-01_events.json
|------------ sub-02_ses-01_task-audiovisual_run-01_events.tsv
|------------ sub-02_ses-01_task-audiovisu

## 生成数据总结

In [31]:
print(mne_bids.make_report(out_path))

Summarizing participants.tsv F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\participants.tsv...
Summarizing scans.tsv files [WindowsPath('F:/Database/Multimodal_data/mne_data/MNE-sample-data/out_data/sample_BIDS/sub-01/ses-01/sub-01_ses-01_scans.tsv'), WindowsPath('F:/Database/Multimodal_data/mne_data/MNE-sample-data/out_data/sample_BIDS/sub-02/ses-01/sub-02_ses-01_scans.tsv')]...
The participant template found: sex were all unknown;
handedness were all unknown;
ages all unknown
This dataset was created by [Unspecified] and conforms to BIDS version 1.7.0.
This report was generated with MNE-BIDS (https://doi.org/10.21105/joss.01896).
The dataset consists of 2 participants (sex were all unknown; handedness were
all unknown; ages all unknown) and 1 recording sessions: 01. Data was recorded
using an MEG system (Elekta) sampled at 600.61 Hz with line noise at 60.0 Hz.
The following software filters were applied during recording:
SpatialCompensation. There were 2 s

## 读取BIDS数据

In [32]:
# 读取刚才生成的BIDS数据
bids_root = pathlib.Path(sample_data_dir / 'out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)

Opening raw data file F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading events from F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv.
Reading channel info from F:\Database\Multimodal_data\mne_data\MNE-sample-data\out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.


In [33]:
raw.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 2560x1351 with 5 Axes>

Channels marked as bad:
['MEG 2443', 'EEG 053']


## 事件被存储为标注，二者可以相互转换

In [34]:
raw.annotations[0]

OrderedDict([('onset', 46.58058882905339),
             ('duration', 0.0),
             ('description', 'Auditory/Right'),
             ('orig_time',
              datetime.datetime(2002, 12, 3, 19, 1, 10, 720100, tzinfo=datetime.timezone.utc))])

In [35]:
events, event_id = mne.events_from_annotations(raw)

Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Left', 'Visual/Right']


In [37]:
# 画出每个事件在时间上的分布
mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])

<Figure size 2560x1448 with 1 Axes>

## 微调MEG精细校准和串扰文件

In [38]:
bids_path.meg_calibration_fpath

WindowsPath('F:/Database/Multimodal_data/mne_data/MNE-sample-data/out_data/sample_BIDS/sub-01/ses-01/meg/sub-01_ses-01_acq-calibration_meg.dat')

In [39]:
bids_path.meg_crosstalk_fpath

WindowsPath('F:/Database/Multimodal_data/mne_data/MNE-sample-data/out_data/sample_BIDS/sub-01/ses-01/meg/sub-01_ses-01_acq-crosstalk_meg.fif')